# Pointings-Search

The pointings-search library is an example library of how we can quickly search for geocentric pointings that would see an object with a given barycentric position. Before running this demo make sure to install the development dependencies of pointing-search `pip install .[dev]`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
from astropy.coordinates import (
    SkyCoord,
    SphericalRepresentation,
    get_body_barycentric,
)
from astropy.time import Time

from pointings_search.pointing_table import PointingTable

## PointingTable

The main data structure for pointings-search is a PointingTable which holds information about the geocentric pointings. Each pointing is given by a RA, dec, and time:

In [ ]:
data_dict = {
    "obsid": [i for i in range(7)],
    "ra": [0.0, 90.0, 45.0, 90.0, 270.0, 223.6095, 223.6095],
    "dec": [0.0, 90.0, 0.0, 45.0, 0.0, -16.6230, -16.6230],
    "obstime": [60261.0, 60281.1, 60241.2, 60211.3, 60291.4, 60253.1, 60353.1],
    "fileloc": [f"./here{i}.txt" for i in range(7)],
}
data = PointingTable.from_dict(data_dict)
print(data.pointings)

We can use helper functions to augment the table with unit vectors indicating the pointing directions and earth's position at each of the observation times.

In [ ]:
data.preprocess_pointing_info()
data.append_earth_pos()
print(data.pointings)

We can plot the various earth positions and pointings in 3-d. The blue dots represent the position of the Earth at the times in the table. The black lines are the pointing vectors. The yellow do is the position of the sun.

We can extend the diagram by including Mars' at MJD=60253.1 as a red dot. We start by using astropy's `get_body_barycentric()` function to compute approximate barycentric position of both Mars in 3-d cartesian space.

In [ ]:
def plot_pointings(ax, data):
    """Plot the 3d pointings on a given axis."""
    for i in range(len(data.pointings)):
        ax.plot(
            [
                data.pointings["earth_vec_x"][i],
                data.pointings["earth_vec_x"][i] + data.pointings["unit_vec_x"][i] * 0.1,
            ],
            [
                data.pointings["earth_vec_y"][i],
                data.pointings["earth_vec_y"][i] + data.pointings["unit_vec_y"][i] * 0.1,
            ],
            [
                data.pointings["earth_vec_z"][i],
                data.pointings["earth_vec_z"][i] + data.pointings["unit_vec_z"][i] * 0.1,
            ],
            color="black",
        )

        # Plot the earth locations
        ax.plot(
            data.pointings["earth_vec_x"][i],
            data.pointings["earth_vec_y"][i],
            data.pointings["earth_vec_z"][i],
            marker=".",
            markersize=5,
            color="blue",
        )

        # Plot the sun
        ax.plot(0, 0, 0, marker=".", markersize=10, color="yellow")


mars_pos_bary = get_body_barycentric("mars", Time(60253.1, format="mjd"))
print(f"Mars' barycentric position {mars_pos_bary}")

ax2 = plt.figure().add_subplot(projection="3d")
plot_pointings(ax2, data)
ax2.plot(
    mars_pos_bary.x.value, mars_pos_bary.y.value, mars_pos_bary.z.value, marker=".", markersize=4, color="red"
)

plt.show()

We can compute the angular distance from the heliocentric viewing directions to the target of interest (3-d in heliocentric space) using vector algebra. First we start by converting Mars' cartesian coordinates into spherical coordinates, then we use `PointingTable`'s `search_heliocentric_pointing()` function.

In [ ]:
ang_dist = data.angular_dist_3d_heliocentric(
    [
        mars_pos_bary.x.value,
        mars_pos_bary.y.value,
        mars_pos_bary.z.value,
    ]
)
print(f"Raw distances (degrees) = {ang_dist}\n")

# Compute Mars' position as viewed from the barycenter.
mars_coords = SkyCoord(SphericalRepresentation.from_cartesian(mars_pos_bary))
print(f"Mars' barycentric coordinates: {mars_coords}\n")

# Define the position in terms of RA, dec, and distance.
mars_vec = SkyCoord(ra=mars_coords.ra, dec=mars_coords.dec, distance=mars_coords.distance)
matches = data.search_heliocentric_pointing(mars_vec, 1.0)
print(matches)

## Proposing Pointings

While the `PointingTable` allows us to compute whether a specific object is in view, it does not help us determine which locations we could search. The `search_algorithms.py` file contains a variety of algorithms that we could use to propose potential search locations.

We start by setting up a limited set of pointings to test.

In [ ]:
ras = []
decs = []
obstimes = []
obstime = 60253.1
for r in np.linspace(45.0, 85.0, 10):
    for d in np.linspace(-55.0, -50.0, 10):
        ras.append(r)
        decs.append(d)
        obstimes.append(obstime)
        obstime += 0.05

data_dict = {
    "ra": ras,
    "dec": decs,
    "obstime": obstimes,
    "fov": [3.0] * len(ras),
    "id": [i for i in range(len(ras))],
}
data = PointingTable.from_dict(data_dict)

### Grid Search (with Distance)

A distance-based grid search will search a uniform grid around the sun with predefined minimum and maximum distance from the baycenter. The red dots represent all the points considered as a potential location and the green dots indicate those which have sufficient number of images (at least 5).

In [ ]:
from pointings_search.search_algorithms import distance_grid_search

results = distance_grid_search(data, num_steps=20, min_images=0, max_results=1e6)

ax3 = plt.figure().add_subplot(projection="3d")
plot_pointings(ax3, data)
for res in results:
    if res[0] >= 5:
        ax3.plot(res[1][0], res[1][1], res[1][2], marker=".", markersize=3, color="green")
    else:
        ax3.plot(res[1][0], res[1][1], res[1][2], marker=".", markersize=1, color="red")
plt.show()

### Grid Search (from projection)

We can do a better job searching by using the geocentric pointings to project fake points into 3-d space and use those to define the bounds of our grid. This focuses the grid on areas of the sky that could actually have a match.

In [ ]:
from pointings_search.search_algorithms import helio_project_grid_search

results = helio_project_grid_search(data, num_steps=10, est_distance=10.0, min_images=0, max_results=1e6)

ax4 = plt.figure().add_subplot(projection="3d")
plot_pointings(ax4, data)
for res in results:
    if res[0] >= 5:
        ax4.plot(res[1][0], res[1][1], res[1][2], marker=".", markersize=3, color="green")
    else:
        ax4.plot(res[1][0], res[1][1], res[1][2], marker=".", markersize=1, color="red")
plt.show()

### Projection Search

Alternately we could just project the geocentric pointings to project fake points into 3-d space and use points directly for our search. This search is particulay useful for large distances.

In [ ]:
from pointings_search.search_algorithms import helio_project_search

results = helio_project_search(data, est_distance=10.0, min_images=0, max_results=1e6)

ax4 = plt.figure().add_subplot(projection="3d")
plot_pointings(ax4, data)
for res in results:
    if res[0] >= 5:
        ax4.plot(res[1][0], res[1][1], res[1][2], marker=".", markersize=3, color="green")
    else:
        ax4.plot(res[1][0], res[1][1], res[1][2], marker=".", markersize=1, color="red")
plt.show()